## Day 12: Subterranean Sustainability

https://adventofcode.com/2018/day/12

### Part 1

Represent the rules as a set. Members of the set are a set of offsets to the index of the current plan that indicate a plant will grow in the next generation. Each generation is represented by a set of indices to pots containing plants. 

In [1]:
from parse import parse


def parse_pots(pots_data):
    pots_string = parse('initial state: {pots}', pots_data)['pots']
    return {i for i, x in enumerate(pots_string) if x == '#'}
    

def parse_rules(rules_data):    
    rules = set([])
    
    for line in rules_data:
        rule = parse('{state} => {outcome}', line)
        
        # If the outcome of a state is a plant then add
        # the offsets to the index of the current plant
        # that contain plants in that state
        if rule['outcome'] == '#':
            rules.add(frozenset(i - 2 
                                for i, x in enumerate(rule['state']) 
                                if x == '#'))
        
    return rules


def parse_input(data):
    lines = [line.strip() for line in data.splitlines()]
    return (parse_pots(lines[0]), parse_rules(lines[2:]))

In [2]:
test_pots, test_rules = parse_input('''initial state: #..#.#..##......###...###

...## => #
..#.. => #
.#... => #
.#.#. => #
.#.## => #
.##.. => #
.#### => #
#.#.# => #
#.### => #
##.#. => #
##.## => #
###.. => #
###.# => #
####. => #''')

In [3]:
test_pots, test_rules

({0, 3, 5, 8, 9, 16, 17, 18, 22, 23, 24},
 {frozenset({-1, 1}),
  frozenset({1, 2}),
  frozenset({-1}),
  frozenset({-1, 1, 2}),
  frozenset({-2, -1, 1}),
  frozenset({-2, -1, 1, 2}),
  frozenset({0}),
  frozenset({-1, 0, 1, 2}),
  frozenset({-2, 0, 1, 2}),
  frozenset({-1, 0}),
  frozenset({-2, -1, 0}),
  frozenset({-2, -1, 0, 2}),
  frozenset({-2, -1, 0, 1}),
  frozenset({-2, 0, 2})})

In [4]:
def generation(rules, pots):
    new_state = set([])
    
    # Need to consider pots two positions either side of current pots
    for pot_i in range(min(pots) - 2, max(pots) + 3):
        offsets = frozenset(i for i in range(-2, 3) if pot_i + i in pots)
        if offsets in rules:
            new_state.add(pot_i)
            
    return new_state


def generations(rules, pots, n):
    for _ in range(n):
        pots = generation(rules, pots)
    return pots


def answer(rules, pots, n=20):
    return sum(generations(rules, pots, n))

In [5]:
answer(test_rules, test_pots)

325

In [6]:
pots, rules = parse_input(open('input', 'r').read())

answer(rules, pots)

3472

### Part 2

Hopefully this will loop.

In [7]:
from itertools import count


def first_loop(rules, pots):
    seen = {frozenset(pots): 0}
    for i in count(1):
        pots = frozenset(generation(rules, pots))
        if pots in seen:
            return (i, seen[pots])
        seen[pots] = i
        
## Don't run this        
# first_loop(rules, pots)

It doesn't look like it, and it's chewing memory. Let's see if the pattern loops with an offset.

In [8]:
def pots_string(pots):
    return ''.join('#' if i in pots else '.' for i in range(min(pots), max(pots) + 1))
    
    
pots_string(test_pots)

'#..#.#..##......###...###'

In [9]:
def first_looped_pattern(rules, pots):
    seen = { pots_string(pots): 0 }
    for i in count(1):
        pots = frozenset(generation(rules, pots))
        s = pots_string(pots)
        if s in seen:
            return (pots, seen[s], i)
        seen[s] = (pots, i)
        
        
next_seen_pots, first_seen, next_seen_time = first_looped_pattern(rules, pots)
first_seen_pots, first_seen_time = first_seen

What's the offset?

In [10]:
min(first_seen_pots), min(next_seen_pots) 

(48, 49)

Just one. How long does it take to loop patterns?

In [11]:
first_seen_time, next_seen_time

(108, 109)

Just one second, which means the pattern of plants is moving forward one second at a time.

In [12]:
fifty_billion = 50000000000

fifty_billionth_pots = (pot + fifty_billion - 108 for pot in first_seen_pots)

In [13]:
sum(fifty_billionth_pots)

2600000000919